<a href="https://colab.research.google.com/github/ancestor9/2025_Winter_Deep-Learning-with-TensorFlow/blob/main/pytorch/Iris_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. 데이터 불러오기
iris = load_iris()
X = iris.data # 4개의 특징 (꽃받침/꽃잎의 길이와 너비)
y = iris.target # 3개의 클래스 (붓꽃 종류)

# 2. 데이터 전처리 (스케일링)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 3. 학습용/테스트용 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. PyTorch 텐서로 변환
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

## Model

In [ ]:
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        # 입력(4) -> 은닉층1(16) -> 은닉층2(16) -> 출력(3)
        self.fc1 = nn.Linear(4, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x) # 마지막은 CrossEntropyLoss 사용을 위해 그대로 출력
        return x

model = IrisNet()

## **Without Batch**

In [ ]:
# 손실 함수와 최적화 도구 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 100번 반복 학습
for epoch in range(100):
    optimizer.zero_grad()    # 기울기 초기화
    outputs = model(X_train) # 예측 (forward)
    loss = criterion(outputs, y_train) # 오차 계산
    loss.backward()          # 역전파 (공부하기)
    optimizer.step()         # 가중치 업데이트

    if (epoch+1) % 20 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')

Epoch [20/100], Loss: 0.3756
Epoch [40/100], Loss: 0.1298
Epoch [60/100], Loss: 0.0590
Epoch [80/100], Loss: 0.0503
Epoch [100/100], Loss: 0.0480


In [ ]:
model.eval() # 평가 모드 전환 (배운 내용!)
with torch.no_grad(): # 기울기 계산 비활성화
    predicts = model(X_test)
    _, predicted_classes = torch.max(predicts, 1) # 가장 높은 점수의 인덱스 찾기

    accuracy = (predicted_classes == y_test).sum().item() / len(y_test)
    print(f'\n최종 테스트 정확도: {accuracy * 100:.2f}%')

# 실제 데이터 한 건 예측해보기
sample = X_test[0].unsqueeze(0) # 1장짜리 박스 포장 (배운 내용!)
sample_pred = model(sample)
print(f'샘플 예측 결과(클래스): {torch.argmax(sample_pred).item()}')
print(f'실제 정답(클래스): {y_test[0].item()}')


최종 테스트 정확도: 100.00%
샘플 예측 결과(클래스): 1
실제 정답(클래스): 1


## **With Batch**

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# 1. 데이터셋 생성 (X_train과 y_train을 하나로 묶음)
train_dataset = TensorDataset(X_train, y_train)

# 2. DataLoader 설정 (배치 크기를 10으로 지정)
# shuffle=True는 매 에폭마다 데이터 순서를 섞어 학습 효율을 높입니다.
train_loader = DataLoader(dataset=train_dataset, batch_size=10, shuffle=True)

# 모델, 손실함수, 최적화 도구는 동일
model = IrisNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 3. 학습 루프 (배치 단위 학습)
for epoch in range(100):
    epoch_loss = 0.0

    # train_loader가 데이터를 10개씩(batch_X, batch_y) 꺼내줍니다.
    for i, (batch_X, batch_y) in enumerate(train_loader):
        optimizer.zero_grad()

        outputs = model(batch_X)     # 10개의 데이터에 대해 예측
        loss = criterion(outputs, batch_y)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    if (epoch+1) % 20 == 0:
        # 평균 손실값 출력
        print(f'Epoch [{epoch+1}/100], Average Loss: {epoch_loss/len(train_loader):.4f}')

Epoch [20/100], Average Loss: 0.0533
Epoch [40/100], Average Loss: 0.0565
Epoch [60/100], Average Loss: 0.0474
Epoch [80/100], Average Loss: 0.0478
Epoch [100/100], Average Loss: 0.0546


In [ ]:
model.eval() # 평가 모드 전환 (배운 내용!)
with torch.no_grad(): # 기울기 계산 비활성화
    predicts = model(X_test)
    _, predicted_classes = torch.max(predicts, 1) # 가장 높은 점수의 인덱스 찾기

    accuracy = (predicted_classes == y_test).sum().item() / len(y_test)
    print(f'\n최종 테스트 정확도: {accuracy * 100:.2f}%')

# 실제 데이터 한 건 예측해보기
sample = X_test[0].unsqueeze(0) # 1장짜리 박스 포장 (배운 내용!)
sample_pred = model(sample)
print(f'샘플 예측 결과(클래스): {torch.argmax(sample_pred).item()}')
print(f'실제 정답(클래스): {y_test[0].item()}')


최종 테스트 정확도: 100.00%
샘플 예측 결과(클래스): 1
실제 정답(클래스): 1


In [ ]:
model.eval()
with torch.no_grad():
    # 실제 데이터 10건 예측해보기
    # X_test에서 처음 10개 샘플을 가져옵니다.
    samples = X_test[:10]
    sample_preds = model(samples)
    _, predicted_classes_10 = torch.max(sample_preds, 1)

    print("\n--- 10개 샘플 예측 결과 ---")
    for i in range(10):
        print(f'샘플 {i+1} 예측 결과(클래스): {predicted_classes_10[i].item()}, 실제 정답(클래스): {y_test[i].item()}')


--- 10개 샘플 예측 결과 ---
샘플 1 예측 결과(클래스): 1, 실제 정답(클래스): 1
샘플 2 예측 결과(클래스): 0, 실제 정답(클래스): 0
샘플 3 예측 결과(클래스): 2, 실제 정답(클래스): 2
샘플 4 예측 결과(클래스): 1, 실제 정답(클래스): 1
샘플 5 예측 결과(클래스): 1, 실제 정답(클래스): 1
샘플 6 예측 결과(클래스): 0, 실제 정답(클래스): 0
샘플 7 예측 결과(클래스): 1, 실제 정답(클래스): 1
샘플 8 예측 결과(클래스): 2, 실제 정답(클래스): 2
샘플 9 예측 결과(클래스): 1, 실제 정답(클래스): 1
샘플 10 예측 결과(클래스): 1, 실제 정답(클래스): 1
